# Use matmul operator of int and float to create range

In MATLAB the for loop looks like:

```
for v = 1.0:-0.2:0.0
   disp(v)
end
```

In Python, we need to use `range()`, and it doesn't support float numbers. Here is the patch code that uses `@` to create loop range object, to simplify the for-loop code:

In [1]:
import cffi
import ctypes
from ctypes import CFUNCTYPE, c_size_t, c_void_p
import sys
from math import ceil

ffi = cffi.FFI()
ctypes.pythonapi.Py_IncRef.argtypes = [ctypes.c_size_t]

class Range:
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
        
    def __matmul__(self, step):
        self.step = step
        return self
        
    def __iter__(self):
        if all(isinstance(v, int) for v in (self.start, self.end, self.step)):
            yield from iter(range(self.start, self.end, self.step))
        else:
            if isinstance(self.step, complex):
                n = int(self.step.imag)
                step = (self.end - self.start) / (n - 1)
            else:
                n = int(ceil((self.end - self.start) / self.step))
                step = self.step
                
            now = self.start
            for i in range(n):
                yield now
                now += step

def nb_matrix_multiply(obj1_addr, obj2_addr):
    obj1 = ctypes.cast(obj1_addr, ctypes.py_object).value
    obj2 = ctypes.cast(obj2_addr, ctypes.py_object).value
    ret_obj = Range(obj1, obj2)
    ctypes.pythonapi.Py_IncRef(id(ret_obj))
    return id(ret_obj)

FUNC = CFUNCTYPE(c_size_t, c_size_t, c_size_t)
cfunc_nb_matrix_multiply = FUNC(nb_matrix_multiply)

def patch_matrix_multiply():        

    offset_tp_as_number = 96
    offset_nb_matrix_multiply = 272    
    
    if sys.maxsize == 0x7fffffff: # if 32bit 
        offset_tp_as_number //= 2
        offset_nb_matrix_multiply //=2
        
    ffi = cffi.FFI()
    for to_patch_type in (int, float):
        tp_as_number_addr = ffi.cast("size_t *", id(to_patch_type) + offset_tp_as_number)
        nb_matrix_multiply_addr = tp_as_number_addr[0] + offset_nb_matrix_multiply
        item = ffi.cast('size_t *', nb_matrix_multiply_addr)
        item[0] = ctypes.cast(cfunc_nb_matrix_multiply, c_void_p).value
        
patch_matrix_multiply()

`5@10` means `range(5, 10)`:

In [2]:
for i in 5@10:
    print(i)

5
6
7
8
9


`2@10@2` means `range(2, 10, 2)`:

In [3]:
for i in 2@10@2:
    print(i)

2
4
6
8


We can use float numbers, `0.0@1.6@0.3` creates the same numbers as `np.arange(0.0, 1.6, 0.3)`:

In [4]:
for x in 0.0@1.6@0.3:
    print(x)

0.0
0.3
0.6
0.8999999999999999
1.2
1.5


`1.2@0@-0.3` is similar to `np.arange(1.2, 0, -0.3)`:

In [5]:
for x in 1.2@0@-0.3:
    print(x)

1.2
0.8999999999999999
0.5999999999999999
0.2999999999999999


If the step is a complex number, the imag part of the number is the total count of the loop, `0@1@5j` is similar to `np.linspace(0, 1, 5)`:

In [6]:
for x in 0@1@5j:
    print(x)

0
0.25
0.5
0.75
1.0


`1@0@5j` is similar to `np.linspace(1, 0, 5)`:

In [7]:
for x in 1@0@5j:
    print(x)

1
0.75
0.5
0.25
0.0


## How it works?

We need to set a function pointer to the field `offset_nb_matrix_multiply` of the field `offset_tp_as_number` of `int` and `float` object:

Here is the code to get field offset:

```python
import cffi
ffi = cffi.FFI()
ffi.cdef("""
size_t offset_tp_as_number;
size_t offset_nb_matrix_multiply;
""")

lib = ffi.verify("""
size_t offset_tp_as_number = offsetof(PyTypeObject, tp_as_number);
size_t offset_nb_matrix_multiply = offsetof(PyNumberMethods, nb_matrix_multiply);
""")

print(lib.offset_tp_as_number)
print(lib.offset_nb_matrix_multiply)
```

The function returns a `Range()` object that uses the two operands as start and end value. `Range` object defines the `__matmul__` method that uses the argument as its step value.